<a href="https://colab.research.google.com/github/jonahlewis4/brain-tumor-recognition/blob/main/Template-Openable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
import tensorflow as tf
import pandas as pd
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
# Set up the directories where your images are stored.
# Make sure to update these paths to where your actual image folders are located.
tumor_dir = 'drive/MyDrive/brain-tumors/archive/yes'       # Directory with brains having tumors
non_tumor_dir = 'drive/MyDrive/brain-tumors/archive/no' # Directory with brains without tumors


# Define the target size for the images (width, height). Feel free to adjust as needed.
target_size = (256, 256)

# Lists to hold the image data and corresponding labels.
data = []
labels = []

current_directory = os.getcwd()
print(current_directory)
print('current direcotry files', os.listdir('./drive'))

def process_images(directory, label):
    """
    Loads images from the specified directory, converts them to greyscale,
    resizes them to the target size, and appends them to the global lists.
    """
    for filename in os.listdir(directory):
        # Check for common image file extensions.
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(directory, filename)
            try:
                # Open the image, convert it to greyscale, and resize it.
                img = Image.open(img_path).convert('L')
                img = img.resize(target_size)
                # Convert the image to a NumPy array.
                img_array = np.array(img)
                data.append(img_array)
                labels.append(label)
            except Exception as e:
                print(f"Could not process {img_path}: {e}")

# Process images from both directories.
process_images(tumor_dir, label=1)      # Label 1 indicates a tumor.
print("this is past opening the tumor directory")
process_images(non_tumor_dir, label=0)  # Label 0 indicates no tumor.
print("this is past opening the non-tumor directory")


# Create a Pandas DataFrame with the image data and labels.
df = pd.DataFrame({
    'image': data,
    'label': labels
})

# Split the DataFrame into training (80%) and testing (20%) sets.
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=65)

print(f"Number of training samples: {len(train_df)}")
print(f"Number of testing samples: {len(test_df)}")

# Preprocess the training and testing data
# Stack the images and reshape to include channel dimension, then normalize.
def prepare_data(df, target_size):
    images = np.stack(df['image'].values)
    images = images.reshape(-1, target_size[0], target_size[1], 1) / 255.0
    labels = np.array(df['label'].values)
    return images, labels

train_images, train_labels = prepare_data(train_df, target_size)
test_images, test_labels = prepare_data(test_df, target_size)

# Using keras we compile a CNN.
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(target_size[0], target_size[1], 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model.
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model.
history = model.fit(train_images, train_labels, epochs=5, batch_size=30, validation_split=0.1)

# Evaluate the model on the test set.
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("Test accuracy:", test_acc)

/content
current direcotry files ['MyDrive', '.shortcut-targets-by-id', '.file-revisions-by-id', '.Trash-0', '.Encrypted']
this is past opening the tumor directory
this is past opening the non-tumor directory
Number of training samples: 202
Number of testing samples: 51


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 32s 4s/step - accuracy: 0.5423 - loss: 0.8349 - val_accuracy: 0.7619 - val_loss: 0.4971
Epoch 2/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 40s 4s/step - accuracy: 0.7795 - loss: 0.5015 - val_accuracy: 0.7619 - val_loss: 0.4523
Epoch 3/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 40s 4s/step - accuracy: 0.8242 - loss: 0.5262 - val_accuracy: 0.7619 - val_loss: 0.4332
Epoch 4/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 43s 4s/step - accuracy: 0.8754 - loss: 0.3624 - val_accuracy: 0.7619 - val_loss: 0.4696
Epoch 5/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 41s 4s/step - accuracy: 0.8628 - loss: 0.3441 - val_accuracy: 0.7619 - val_loss: 0.4448
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 682ms/step - accuracy: 0.7676 - loss: 0.5058
Test accuracy: 0.7450980544090271
